In [6]:
import pandas as pd
import numpy as np
import os

#viz
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import plotly

In [5]:
#import data
df = pd.read_csv(r'C:\Users\pc\Desktop\PROJECTS\DATA_PRODUCT\clean_file.csv', delimiter=',')
df.shape

(8133, 24)

In [7]:
#we only have natural gas left after mining
print (df['Fuel Type Code'].unique())

['ELEC' 'E85' 'BD' 'CNG' 'LPG' 'LNG' 'HY']
